In [1]:
import copy
import lxml.html
from pathlib import Path
import pickle
import re
import time
from tqdm import tqdm
from stanza.server import CoreNLPClient
from projectmir.xmldocument import XMLDocument
from projectmir.extract_definition import pattern_based_extract_description

In [2]:
p_path = './data/test_latexml/'
p = Path(p_path)
x = list(p.glob('*.html'))
documents_IDs = [x_.name[:-5] for x_ in x]
docs_list = [[]]*len(documents_IDs)
for i, documents_ID in tqdm(enumerate(documents_IDs)):
    document_path = p_path + documents_ID + '.html'
    start_time = time.time()
    docs_list[i] = XMLDocument(document_path)
save_filename = 'docs_list' + time.strftime('_%d%b%Y_%H%M%S') + '.pkl'
with open(save_filename, 'wb') as f:
    pickle.dump(docs_list, f)

0it [00:00, ?it/s]2020-08-24 21:25:57 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |

2020-08-24 21:25:57 INFO: Use device: cpu
2020-08-24 21:25:57 INFO: Loading: tokenize
2020-08-24 21:25:57 INFO: Done loading processors!
2020-08-24 21:25:58 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | ewt       |
| pos       | ewt       |
| lemma     | ewt       |
| depparse  | ewt       |
| ner       | ontonotes |

2020-08-24 21:25:58 INFO: Use device: cpu
2020-08-24 21:25:58 INFO: Loading: tokenize
2020-08-24 21:25:58 INFO: Loading: pos
2020-08-24 21:25:59 INFO: Loading: lemma
2020-08-24 21:25:59 INFO: Loading: depparse
2020-08-24 21:26:00 INFO: Loading: ner
2020-08-24 21:26:00 INFO: Done loading processors!
1it [00:50, 50.26s/it]2020-08-24 21:26:47 INFO: Loading these models for language: en (English):
| Processor | Package |
------------------

loaded data.
processing data...
preprocessed document
extract identifiers...
Number of math components is 50
POS tagging...
extract candidate definition...
Starting server with command: java -Xmx16G -cp /Users/kato/GoogleDrive/project-mir/data/resources/stanford-corenlp-4.1.0/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 30000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-c2bcbb6d480140a3.props -preload tokenize,ssplit,pos,lemma,ner,depparse
compute the properties of candidates...
elapsed time: 50.26129 seconds ---
loaded data.
processing data...
preprocessed document
extract identifiers...
Number of math components is 50
POS tagging...
extract candidate definition...
Starting server with command: java -Xmx16G -cp /Users/kato/GoogleDrive/project-mir/data/resources/stanford-corenlp-4.1.0/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 30000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp

In [ ]:
# # load_filename = 'docs_list.pkl'
# load_filename = 'docs_list_24Aug2020_203418.pkl'
# with open(load_filename, 'rb') as f:
#     docs_list = pickle.load(f)

In [3]:
%%time

doc0 = docs_list[0]
print(doc0.path)

# extract definitions using pattern based method.
pattern_based_definition_list = []
for i, identifier in enumerate(doc0.identifiers):
    definition = pattern_based_extract_description(identifier)
    pattern_based_definition_list.append(definition)


./data/test_latexml/test_sample.html
CPU times: user 508 µs, sys: 78 µs, total: 586 µs
Wall time: 568 µs


In [4]:
doc0.compute_candidate_statistics()

In [6]:
for i, identifier_ in enumerate(doc0.identifiers):
    print(identifier_.sentences[0].original)
    print('===============================')
    for j, candidate_ in enumerate(identifier_.candidates):
        print(candidate_.text)
        print(candidate_.included_sentence)
        print(doc0.identifiers[i].candidates[j].word_count_btwn_var_cand,
              doc0.identifiers[i].candidates[j].score_match_character,
              doc0.identifiers[i].candidates[j].candidate_count_in_sentence)

The liquid inlet stream consists of a single component with a mass flow rate MATH0000 and an inlet temperature MATH0001 .
The liquid inlet stream
CANDIDATE consists of a single component with a mass flow rate MATH0000 and an inlet temperature MATH0001
10 1 1
a single component
The liquid inlet stream consists of CANDIDATE with a mass flow rate MATH0000 and an inlet temperature MATH0001
5 0 1
a mass flow rate
The liquid inlet stream consists of a single component with CANDIDATE MATH0000 and an inlet temperature MATH0001
0 0 1
an inlet temperature MATH0001
The liquid inlet stream consists of a single component with a mass flow rate MATH0000 and CANDIDATE
1 1 1
The liquid inlet stream consists of a single component with a mass flow rate MATH0000 and an inlet temperature MATH0001 .
The liquid inlet stream
CANDIDATE consists of a single component with a mass flow rate MATH0000 and an inlet temperature MATH0001
15 2 1
a single component with a mass flow rate MATH0000 and an inlet temperature

In [ ]:
for i, identifier_ in enumerate(doc0.identifiers[:1]):
    for j, candidate_ in enumerate(identifier_.candidates):
        math_txt = f'MATH{i:04d}'
        sentence_list = candidate_.included_sentence.split()
        sentence_list = [s_.rstrip(',. :;') for s_ in sentence_list]
        math_txt_index = [i for i, term in enumerate(sentence_list) if term == math_txt]
        candidate_index = [i for i, term in enumerate(sentence_list) if term == 'CANDIDATE']
        word_count_btwn_var_cand = len(sentence_list)
        for math_txt_index_ in math_txt_index:
            for candidate_index_ in candidate_index:
                word_count_btwn_var_cand = \
                    min(word_count_btwn_var_cand,
                        abs(math_txt_index_ - candidate_index_) - 1)
        doc0.identifiers[i].candidates[j].word_count_btwn_var_cand = word_count_btwn_var_cand



